Notebook to look at encoding models of A00c Activity; here we consider all A00c cells together (without regard for where manipulations were delivered or cell identity)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import pickle

from janelia_core.stats.regression import grouped_linear_regression_boot_strap
from janelia_core.stats.regression import visualize_boot_strap_results

from keller_zlatic_vnc.linear_modeling import one_hot_from_table
from keller_zlatic_vnc.linear_modeling import color_grp_vars

In [ ]:
%matplotlib qt

In [ ]:
# Helps make sure our plots will have the same size
plt.rcParams['figure.constrained_layout.use'] = True

## Parameters go here

In [ ]:
data_dir = r'/Users/williambishop/Desktop'
data_file = 'A00c_activity.pkl'

# Behavior before and after consider
beh_before = ['Q', 'F', 'B']
beh_after = ['Q', 'F', 'B']
interaction_terms = True

# How many boot strap samples we use in each analysis
n_bs_smps = 1000

# Determine how we will display results
font_size = 15
alpha = .01

# Determine what type of manipulation events we look at
manip_type = 'both' # 'both', 'A4' or 'A9'

## Load the data

In [ ]:
data_path = Path(data_dir) / data_file
with open(data_path, 'rb') as f:
    data = pickle.load(f)
data = pd.DataFrame(data)

## Down select to only the manipulation events we want to consider

In [ ]:
if manip_type == 'A4':
    print('Analyzing only A4 manipulation events.')
    data = data[data['man_tgt'] == 'A4']
elif manip_type == 'A9':
    print('Analyzing only A9 manipulation events.')
    data = data[data['man_tgt'] == 'A9']
else:
    print('Analyzing all manipulation events.')

## Get groups of data (a group corresponds to each subject)

In [ ]:
unique_ids = data['subject_id'].unique()
g = np.zeros(len(data))
for u_i, u_id in enumerate(unique_ids):
    g[data['subject_id'] == u_id] = u_i

## Pull out $\Delta F / F$

In [ ]:
dff_before = data['dff_before'].to_numpy()
dff_after = data['dff_after'].to_numpy()

## First, we look at linear models without accounting for subject identity of manipulation type

### Perform the regression and bootstrap computations

In [ ]:
basic_enc_data, basic_enc_vars = one_hot_from_table(data, beh_before=beh_before, beh_after=beh_after,
                                        enc_subjects=False, enc_beh_interactions=interaction_terms) 
basic_enc_vars.append('mean')

In [ ]:
n_beh_cols = len(beh_before) + len(beh_after) + interaction_terms*len(beh_before)*len(beh_after)
basic_keep_rows =  np.logical_not(np.all(basic_enc_data[:, 0:n_beh_cols] == 0, axis=1))

basic_enc_data = basic_enc_data[basic_keep_rows, :]
basic_dff_before = dff_before[basic_keep_rows]
basic_dff_after = dff_after[basic_keep_rows]
basic_g = g[basic_keep_rows]


In [ ]:
basic_dff_before_bs_results = grouped_linear_regression_boot_strap(x=basic_enc_data, 
                                                                   y=basic_dff_before, 
                                                                   g=basic_g, 
                                                                   n_bs_smps=n_bs_smps, 
                                                                   include_mean=True)

In [ ]:
basic_dff_after_bs_results = grouped_linear_regression_boot_strap(x=basic_enc_data, 
                                                                  y=basic_dff_after, 
                                                                  g=basic_g, 
                                                                  n_bs_smps=n_bs_smps, 
                                                                  include_mean=True)

### Display results

In [ ]:
ax = visualize_boot_strap_results(bs_values=basic_dff_before_bs_results[0], 
                                  var_strs=basic_enc_vars, 
                                  theta = basic_dff_before_bs_results[1],
                                  var_clrs=color_grp_vars(basic_enc_vars),
                                  violin_plots=True,
                                  alpha=alpha)

ph = plt.ylabel('Coef Vls', fontsize=font_size)
ax.tick_params(axis="x", labelsize=font_size)
ax.tick_params(axis="y", labelsize=font_size)

title = 'Encoding for Pre Manipulation $\Delta F /F$ Activity'
if manip_type == 'both':
    title = title + ', both A4 & A9'
else:
    title = title + ', ' + manip_type

plt.title(title, fontsize=font_size, pad=50)
fig = plt.gcf()
fig.set_size_inches(12, 8)


In [ ]:
ax = visualize_boot_strap_results(bs_values=basic_dff_after_bs_results[0], 
                                  var_strs=basic_enc_vars, 
                                  theta = basic_dff_after_bs_results[1],
                                  var_clrs=color_grp_vars(basic_enc_vars),
                                  violin_plots=True,
                                  alpha=alpha)

ph = plt.ylabel('Coef Vls', fontsize=font_size)
ax.tick_params(axis="x", labelsize=font_size)
ax.tick_params(axis="y", labelsize=font_size)

title = 'Encoding for Post Manipulation $\Delta F /F$ Activity'

if manip_type == 'both':
    title = title + ', both A4 & A9'
else:
    title = title + ', ' + manip_type
    
plt.title(title, fontsize=font_size, pad=50)
fig = plt.gcf()
fig.set_size_inches(12, 8)

### Look at correlation of regressors 

In [ ]:
#plt.imshow(np.corrcoef(basic_enc_data.transpose()), cmap='PiYG', vmin=-1, vmax=1)
#plt.colorbar()

## Now we account for subject identity

### Perform regression and bootstrap computations

In [ ]:
sub_enc_data, sub_enc_vars = one_hot_from_table(data, beh_before=beh_before, beh_after=beh_after,
                                        enc_subjects=True, enc_beh_interactions=interaction_terms) 

In [ ]:
sub_keep_rows =  np.logical_not(np.all(sub_enc_data[:, 0:n_beh_cols] == 0, axis=1))

sub_enc_data = sub_enc_data[sub_keep_rows, :]
sub_dff_before = dff_before[sub_keep_rows]
sub_dff_after = dff_after[sub_keep_rows]
sub_g = g[sub_keep_rows]

In [ ]:
sub_dff_before_bs_results = grouped_linear_regression_boot_strap(x=sub_enc_data, 
                                                                   y=sub_dff_before, 
                                                                   g=sub_g, 
                                                                   n_bs_smps=n_bs_smps, 
                                                                   include_mean=False)

In [ ]:
sub_dff_after_bs_results = grouped_linear_regression_boot_strap(x=sub_enc_data, 
                                                                  y=sub_dff_after, 
                                                                  g=sub_g, 
                                                                  n_bs_smps=n_bs_smps, 
                                                                  include_mean=False)

In [ ]:
len(np.unique(sub_g))

### Display results

In [ ]:
ax = visualize_boot_strap_results(bs_values=sub_dff_before_bs_results[0][:, 0:n_beh_cols], 
                                  var_strs=sub_enc_vars[0:n_beh_cols], 
                                  theta = sub_dff_before_bs_results[1][0:n_beh_cols],
                                  var_clrs=color_grp_vars(sub_enc_vars[0:n_beh_cols]),
                                  violin_plots=True,
                                  alpha=alpha, 
                                  sig_y_vl=-.18)


ph = plt.ylabel('Coef Vls', fontsize=font_size)
ax.tick_params(axis="x", labelsize=font_size)
ax.tick_params(axis="y", labelsize=font_size)
ax.set_ylim([-.2, .2])

title = 'Encoding for Pre Manipulation $\Delta F /F$ Activity'
if manip_type == 'both':
    title = title + ', both A4 & A9'
else:
    title = title + ', ' + manip_type

plt.title(title, fontsize=font_size, pad=50)
fig = plt.gcf()
fig.set_size_inches(12, 8)

In [ ]:
ax = visualize_boot_strap_results(bs_values=sub_dff_after_bs_results[0][:, 0:n_beh_cols], 
                                  var_strs=sub_enc_vars[0:n_beh_cols], 
                                  theta = sub_dff_after_bs_results[1][0:n_beh_cols],
                                  var_clrs=color_grp_vars(sub_enc_vars[0:n_beh_cols]),
                                  violin_plots=True,
                                  alpha=alpha, 
                                  sig_y_vl = -.9)

ph = plt.ylabel('Coef Vls', fontsize=font_size)

ax.tick_params(axis="x", labelsize=font_size)
ax.tick_params(axis="y", labelsize=font_size)
ax.set_ylim([-1, 1])

title = 'Encoding for Post Manipulation $\Delta F /F$ Activity'

if manip_type == 'both':
    title = title + ', both A4 & A9'
else:
    title = title + ', ' + manip_type
    
plt.title(title, fontsize=font_size, pad=50)
fig = plt.gcf()
fig.set_size_inches(12, 8)

### Look at correlation of regressors

In [ ]:
plt.imshow(np.corrcoef(sub_enc_data.transpose()), cmap='PiYG', vmin=-1, vmax=1)
plt.colorbar()